## Задачи исследования
1. Отличаются ли пелены от бинтов по параметрам?
2. Отличаются ли узкие, средние и широкие бинты друг от друга по параметрам?
3. Каков процент рогожки?
4. Какие группы тканей по качеству получаются?
5. Различаются ли нити основы и нити утка?
6. Какое количество (процент) тканей высокого качества имеют сбалансированное переплетение?
7. Какое количество (процент) тканей среднего качества имеет сбалансированное переплетение?
8. Какое количество (процент) тканей ниского качества имеет сбалансированное переплетение?
9. Каково соотношение сбалансированных тканей и тканей с преобладанием нитей основы?
10. Высчитать "коэффициент заполняемости" тканей
11. Разделить ткани по сбалансированности (сбалансированные - ±3 нити; ткани с преобладанием нитей основы); рассчитать процент каждой группы

In [1]:
import pandas
import pandas_profiling
import seaborn
import scipy.stats as stats
import matplotlib.pyplot as pyplot
%matplotlib inline

import modules.textiles as textile

In [2]:
textiles = textile.Textiles('dataframe.csv')
outliers = textile.TextileOutliers(textiles)

# # Ткани с переплетением типа "рогожка";
# baskets = dataframe[dataframe['weaving_technique'] == 'basket']

# # выборка узких бинтов;
# narrow_bands = dataframe[dataframe['type'] == 'n_band']

# # выборка средних бинтов;
# middle_bands = dataframe[dataframe['type'] == 'm_band']

# # выборка широких бинтов;
# wide_bands = dataframe[dataframe['type'] == 'w_band']

NameError: name 'warp_mean_quantile_3' is not defined

In [5]:
# report = pandas_profiling.ProfileReport(textiles.dataframe())
# report.get_description()

### Описание датафрейма
1. 'internal_storage' - номер по каталогу
2. 'grave' - номер могилы
3. 'warp_size' - размер изделия по основе
4. 'weft_size' - размер изделия по утку
5. 'type' - тип изделия
6. 'condition' - сохранность изделия
7. 'weaving_technique'
8. 'warp_material' - материал нитей основы
9. 'weft_material' - материал нитей утка
10. 'warp_dyed' - окрашивание нитей основы
11. 'weft_dyed' - окрашивание нитей утка
12. 'warp_twist' -  крутка нитей основы
13. 'weft_twist' - крутка нитей утка
14. 'warp_angle'
15. 'weft_angle'
16. 'warp_a' - толщина нитей основы (нижний предел)
17. 'warp_b' - толщина нитей основы (верхний предел)
18. 'weft_a' - толщина нитей утка (нижний предел)
19. 'weft_b' - толщина нитей утка (верхний предел)
20. 'warp_dens' - плотность по основе
21. 'weft_dens' - плотность по утку  
* * *
Числовые параметры - warp_size, wef_size, warp_a, warp_b, weft_a, weft_b, warp_mean, weft_mean, warp_dens, weft_dens

In [6]:
dataframe = textiles.dataframe()
dataframe.head()

,grave,internal_storage,warp_size,weft_size,type,condition,weaving_technique,warp_material,weft_material,warp_dyed,...,angle_weft,warp_a,warp_b,weft_a,weft_b,warp_dens,weft_dens,warp_mean,weft_mean,density_coefficient
0,80,2007-0075-002,100,110,fabric,fragment,tabby,linen,linen,no,...,medium,0.5,1.0,0.5,1.0,13.0,8.0,0.75,0.75,1.62
1,80,2007-0075-003,290,40,n_band,fragment,tabby,linen,linen,no,...,medium,0.3,0.5,0.6,0.8,16.0,10.0,0.40,0.70,1.60
2,86,2007-0056-003,140,90,fabric,fragment,tabby,linen,linen,no,...,-,0.5,0.5,0.1,0.2,18.0,6.0,0.50,0.15,3.00
3,86,2007-0056-004,300,20,n_band,fragment,tabby,linen,linen,no,...,medium,0.2,0.5,0.2,0.6,30.0,9.0,0.35,0.40,3.33
4,87,2007-0059-001,420,20,n_band,fragment,tabby,linen,linen,no,...,medium,0.4,0.6,0.5,0.6,24.0,10.0,0.50,0.55,2.40


In [7]:
densities = dataframe[['warp_dens', 'weft_dens']]
densities_correlation = densities.corr(method='spearman')

thick = dataframe[['warp_mean', 'weft_mean']]
thick_corr = thick.corr(method='spearman')

print('Корреляция плотностей по основе и по утку')
print(densities_correlation)
print()
print('Корреляция толщин нитей по основе и по утку')
print(thick_corr)

Корреляция плотностей по основе и по утку
           warp_dens  weft_dens
warp_dens   1.000000   0.563466
weft_dens   0.563466   1.000000

Корреляция толщин нитей по основе и по утку
           warp_mean  weft_mean
warp_mean   1.000000   0.671606
weft_mean   0.671606   1.000000


### #1 Отличаются ли пелены от бинтов по параметрам?

In [31]:
shrouds = dataframe[dataframe['type'] == 'funeral_shroud']
print(len(shrouds))
bands = dataframe.loc[dataframe['type'].isin(['n_band', 'm_band', 'w_band'])]
print(len(bands))

25
194


### #2 Отличаются ли узкие, средние и широкие бинты друг от друга по параметрам?

In [11]:
narrow_bands = dataframe[dataframe.type == 'n_band']
middle_bands = dataframe[dataframe.type == 'm_band']
wide_bands = dataframe[dataframe.type == 'w_band']

### #5 Различаются ли нити основы и нити утка?

### #11 Разделить ткани по сбалансированности (сбалансированные - ±3 нити; ткани с преобладанием нитей основы); рассчитать процент каждой группы

In [8]:
balanced = textiles.balanced()
unbalanced = textiles.unbalanced()
print('Сбалансированных тканей - ' + str(len(balanced)))
print('Несбалансированных тканей - ' + str(len(unbalanced)))

Сбалансированных тканей - 47
Несбалансированных тканей - 256
